In [ ]:
# default_exp data.transforms

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Transforms
> Utilities for image transforms.

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
from albumentations import (RandomRotate90,
                            Flip,
                            Transpose,
                            GridDistortion,
                            RandomCrop,
                            GaussianBlur,
                            RandomGamma,
                            RandomBrightnessContrast,
                            HueSaturationValue,
                            RGBShift,
                            CenterCrop)

In [ ]:
# export
class DeterministicHSV(HueSaturationValue):
    def __init__(self, num_els=1, **kwargs):
        super().__init__(**kwargs)
        self.always_apply = True
        self.num_els = num_els
        self.base_values = super().get_params()
        self.max_values = {"hue_shift": self.hue_shift_limit,
                           "sat_shift": self.sat_shift_limit,
                           "val_shift": self.val_shift_limit}
        self.n = 0
        self.mult = np.random.randint(10000)

    def get_params(self):
        params = {}
        for k, v in self.base_values.items():
            v_min, v_max = self.max_values[k]
            p = v + self.n * self.mult
            p = (p - v_min) % (v_max - v_min) + v_min
            params[k] = p
        self.n += 1
        self.n %= self.num_els
        return params

In [ ]:
# export
class DeterministicBrightnessContrast(RandomBrightnessContrast):
    def __init__(self, num_els=1, **kwargs):
        super().__init__(**kwargs)
        self.always_apply = True
        self.num_els = num_els
        self.base_values = super().get_params()
        self.max_values = {"alpha": tuple(x + 1 for x in self.contrast_limit),
                           "beta": self.brightness_limit}
        self.n = 0
        self.mult = np.random.randint(10000)    
    
    def get_params(self):
        params = {}
        for k, v in self.base_values.items():
            v_min, v_max = self.max_values[k]
            p = v + self.n * self.mult
            p = (p - v_min) % (v_max - v_min) + v_min
            params[k] = p
        self.n += 1
        self.n %= self.num_els
        return params

In [ ]:
# export
def get_transforms1(size, num_els=1):
    tfms = [RandomRotate90(),
            Flip(),
            Transpose(),
            GridDistortion(distort_limit=0.05, p=0.2),
            RandomGamma(p=0.2),
            GaussianBlur(blur_limit=3, p=0.2),
            RandomCrop(size, size)]
    val_tfms = [CenterCrop(size, size)]
    return tfms, val_tfms

Return a tuple `(train_tfms, val_tfms)` containing transforms defined in [`albumentations`](https://albumentations.readthedocs.io) for training and validation.

Transforms for training are :
* [`RandomRotate90`](https://albumentations.readthedocs.io/en/latest/api/augmentations.html#albumentations.augmentations.transforms.RandomRotate90)
* [`Flip`](https://albumentations.readthedocs.io/en/latest/api/augmentations.html#albumentations.augmentations.transforms.Flip)
* [`Transpose`](https://albumentations.readthedocs.io/en/latest/api/augmentations.html#albumentations.augmentations.transforms.Transpose)
* [`GridDistortion`](https://albumentations.readthedocs.io/en/latest/api/augmentations.html#albumentations.augmentations.transforms.GridDistortion)
* [`RandomGamma`](https://albumentations.readthedocs.io/en/latest/api/augmentations.html#albumentations.augmentations.transforms.RandomGamma)
* [`GaussianBlur`](https://albumentations.readthedocs.io/en/latest/api/augmentations.html#albumentations.augmentations.transforms.GaussianBlur)

Both training and validation contain [`RandomCrop`](https://albumentations.readthedocs.io/en/latest/api/augmentations.html#albumentations.augmentations.transforms.RandomCrop).

In [ ]:
# export
def get_transforms2(size, num_els=1):
    tfms = [RandomRotate90(),
            Flip(),
            Transpose(),
            GridDistortion(distort_limit=0.05, p=0.2),
            RandomGamma(p=0.2),
            GaussianBlur(blur_limit=3, p=0.2),
            RGBShift(0.15, 0.15, 0.15),
            RandomCrop(size, size)]
    val_tfms = [CenterCrop(size, size)]
    return tfms, val_tfms

In [ ]:
# export
def get_transforms3(size, num_els=1):
    tfms = [RandomRotate90(),
            Flip(),
            Transpose(),
            GridDistortion(distort_limit=0.05, p=0.2),
            RandomBrightnessContrast(p=0.7),
            GaussianBlur(blur_limit=3, p=0.2),
            RGBShift(0.2, 0.2, 0.2, p=0.8),
            RandomCrop(size, size)]
    val_tfms = [RandomBrightnessContrast(p=0.7),
                RGBShift(0.2, 0.2, 0.2, p=0.7),
                CenterCrop(size, size)]
    return tfms, val_tfms

In [ ]:
# export
def get_transforms4(size, num_els=1):
    tfms = [RandomRotate90(),
            Flip(),
            Transpose(),
            GridDistortion(distort_limit=0.05, p=0.2),
            RandomBrightnessContrast(0.1, 0.1, p=0.7),
            GaussianBlur(blur_limit=3, p=0.2),
            HueSaturationValue(30, 0., 0., p=0.8),
            RandomCrop(size, size)]
    val_tfms = [DeterministicBrightnessContrast(num_els=num_els, brightness_limit=0.1, contrast_limit=0.1),
                DeterministicHSV(num_els=num_els, hue_shift_limit=30, sat_shift_limit=0., val_shift_limit=0.),
                CenterCrop(size, size)]
    return tfms, val_tfms

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_train.ipynb.
Converted 02_predict.ipynb.
Converted 10_data.read.ipynb.
Converted 11_data.loaders.ipynb.
Converted 12_data.dataset.ipynb.
Converted 13_data.utils.ipynb.
Converted 14_data.transforms.ipynb.
Converted 20_models.plmodules.ipynb.
Converted 21_models.modules.ipynb.
Converted 22_models.utils.ipynb.
Converted 23_models.hooks.ipynb.
Converted 24_models.metrics.ipynb.
Converted 25_models.losses.ipynb.
Converted 80_params.defaults.ipynb.
Converted 81_params.parser.ipynb.
Converted 99_index.ipynb.
